In [1]:
# this cell provides some custom styling for the markdown cells in this notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)

css_styling();

# GC-MS Data Processing and Analysis

This notebook performs the processing and analysis of the data contained in the `/gcms_data` folder.

## Summary of Procedure

We are given the prompt that we need to analyze three gasoline samples and determine the concentration of benzene, toluene, and methyl *t*-butyl ether (MTBE) in each. In order to accomplish this, we will use Gas Chromatography coupled with Mass Spectrometry (GC-MS). Calibration curves for each of the analytes will be made using standard solutions with known concentrations. An internal standard of deuterated toluene (d8-toluene) with a fixed concentration will be used for all solutions, including the gasoline samples.

### Internal Standard Stock Solution

100 $\mu L$ of d8-toluene is diluted to a total volume of 6.00 $mL$ with 1-chlorohexadecane.

### Analyte Standard Stock Solutions

Three standard stock solutions are prepared according to the following table:

<table style="margin-left: 20px">
    <thead>
        <tr style="border-top: 1px solid;"><th></th><th class="center" colspan="3">volume [$\mu L$]</th></tr>
        <tr style="border-bottom: 1px solid;">
            <th class="center">compound</th>
            <th class="center">standard 1</th>
            <th class="center">standard 2</th>
            <th class="center">standard 3</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td class="center">MTBE</td>
            <td class="right">50</td>
            <td class="right">20</td>
            <td class="right">10</td>
        </tr>
        <tr>
            <td class="center">toluene</td>
            <td class="right">200</td>
            <td class="right">100</td>
            <td class="right">50</td>
        </tr>
        <tr>
            <td class="center">benzene</td>
            <td class="right">50</td>
            <td class="right">20</td>
            <td class="right">10</td>
        </tr>
        <tr>
            <td class="center">1-chlorohexadecane</td>
            <td class="right">2200</td>
            <td class="right">2360</td>
            <td class="right">2430</td>
        </tr>
    </tbody>
</table>

### Sample Preparation for GC-MS

The final samples are prepared by combining 20 $\mu L$ of a given standard stock solution with 1.00 $mL$ of 1-chlorohexadecane and 50 $\mu L$ of the d8-toluene stock solution, for a total sample volume of 1070 $\mu L$. Each standard is prepared in triplicate, and each solution will be run three times.

The gasoline samples are prepared in the same manner, by combining 20 $\mu L$ of the raw gasoline sample with 1.00 $mL$ of 1-chlorohexadecane and 50 $\mu L$ of the d8-toluene stock solution. Only one sample is prepared for each gasoline sample, and each sample is run three times.

1 $\mu L$ of each sample is injected into the GC-MS for each run.

### Data

**Raw Data**

Examples of the raw data collected from the GC-MS runs are contained in the `/gcms_data/raw_data` folder. `C153AC1A.D`, `C153AC2A.D`, and `C153AC3A.D` are examples for Standard 1, Standard 2, and Standard 3, respectively. `C153AG1.D`, `C153AG2.D`, and `C153AG3.D` correspond to three different gasoline samples.

**Report Summaries**

The `/gcms_data/integration_reports` folder contains Integration Reports for the standard and gasoline solutions in CSV format.

---

## Data Processing and Analysis

### Creating Calibration Curves from the Standard Data

In order to quantify the concentrations of the analytes in the gasoline samples, we need to use the data from the standard solutions to create calibration curves relating molarity to peak area. Since we are provided the Integration Report data, we can start from there.

#### Reading In The Integration Report

The first thing we need to do is tell Python where the data files are stored on our computer. To do this, we will make use of the built-in `pathlib` library. This library allows us to create a `Path` object that Python can use to find files and read their contents later.

For more information on working with paths and files in Python, check out [this excellent course](https://realpython.com/read-write-files-python/) from [Real Python](https://realpython.com/)

In [2]:
from pathlib import Path

In [3]:
# Path to the integration report data
# the ../../ tells the computer to go two folder up

REPORTS_FOLDER = Path('../../gcms_data/integration_reports')

STANDARDS_FILE = REPORTS_FOLDER / 'Integration_Report_Standards.csv'
GASOLINE_FIL = REPORTS_FOLDER / 'Integration_Report_Gasoline.csv'

Next, we will use the common data analysis library <a href="https://pandas.pydata.org/"><img src="https://pandas.pydata.org/static/img/pandas_white.svg" alt="Pandas" width="75" /></a> to work with and manipulate the Integration Report data.

Pandas uses `DataFrames` to store data. These function similarly to an Excel spreadsheet table in that the data are stored in columns and rows.

In [4]:
import pandas as pd

In [5]:
# Create a DataFrame from the STANDARDS FILE
standard_report = pd.read_csv(STANDARDS_FILE)

# Display the 'head' (first 5 rows) of the DataFrame
standard_report.head()

,Standards,Extracted Ion Mass (-0.3 +0.7),Retention Time,Area
0,Standard1_Vial1_Run1,73.0,1.313,13223058.0
1,NaN,91.0,3.281,62865953.0
2,NaN,78.0,1.842,17074346.0
3,NaN,100.0,3.200,17146218.0
4,NaN,NaN,NaN,NaN


#### Cleaning the Data

We can see that while pandas was able to easily read the data stored in the `Integration_Report_Standards.csv` file, the table that was produced is a bit messy. First of all, there are empty rows between each sample run that have been included in the dataframe. We can easily get rid of these by running the `dropna()` function.

In [6]:
# remove all rows that only contain `NaN` values (empty rows in the CSV)
standard_report.dropna(axis=0, how="all", inplace=True)

Next, we can see that the `Standards` column only lists the sample name once for a given run. While it's easy for a human to interpret this format, we should explicitly label each row as belonging to a certain standard, vial, and run. Luckily, there's another easy function, `fillna()`, that allows us to fill in these values. We will specify the "forward fill" method, which tells the function to replace any `NaN` values with the value of the cell above it.

In [7]:
# use the 'forward fill' (`ffill`) method to replace the NaN values for the sample names
standard_report.fillna(method='ffill', inplace=True)

Great! We can now begin to work with the data - everything has a label, and there are no missing values from the table.

While we could begin to write some functions to extract the values we need to begin constructing our calibration curves, it will be easier in the long run if we "pivot" the table so that the retention times and areas are the columns, with each sample run as an index. To accomplish this, we'll make a new dataframe that uses the values in `standard_report`.

In [8]:
# create a new dataframe by pivoting on the extracted ion mass column
standard_summary = standard_report.pivot(index="Standards", columns="Extracted Ion Mass (-0.3 +0.7)", values=["Retention Time", "Area"])
standard_summary.head()

Retention Time                       \
Extracted Ion Mass (-0.3 +0.7)          73.0   78.0   91.0   100.0   
Standards                                                            
Standard1_Vial1_Run1                    1.313  1.842  3.281  3.200   
Standard1_Vial1_Run2                    1.331  1.858  3.299  3.217   
Standard1_Vial1_Run3                    1.336  1.865  3.306  3.223   
Standard1_Vial2_Run1                    1.319  1.850  3.291  3.208   
Standard1_Vial2_Run2                    1.303  1.831  3.274  3.191   

                                      Area                                      
Extracted Ion Mass (-0.3 +0.7)       73.0        78.0        91.0        100.0  
Standards                                                                       
Standard1_Vial1_Run1            13223058.0  17074346.0  62865953.0  17146218.0  
Standard1_Vial1_Run2            13936945.0  17363726.0  77036137.0  17822874.0  
Standard1_Vial1_Run3            13233195.0  17442001.0  72660448.0  18128977.0  
Standard1_Vial2_Run1            14306901.0  16930185.0  67847488.0  17569413.0  
Standard1_Vial2_Run2            13597888.0  16764926.0  66299143.0  17927930.0

This first step of pivoting the data has moved us in the right direction - the data are now arranged in columns with each ion mass as the header, and the retention times and areas for each ion as the data. However, the format of the columns is now a `MultiIndex` (read more [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)). You can verify this for yourself by calling `standard_summary.columns` if you like.

In order to make the data easier to access, we need to "flatten" the column labels. We can do this, while simultaneously renaming the columns using the fairy complicated one-liner below:

In [9]:
# This line of code uses a lot of Python tricks, including f-strings, list comprehension, string methods, and tuple unpacking.
standard_summary.columns = [f"{int(B)}_{A}".replace(" ", "_").replace("Retention_Time", "RT") for A, B in standard_summary.columns.values]

As a final step before moving on to actually analyzing the data and constructing our normalized calibration curves, we will change the datatype of the Area columns from `float64` to `int64`. Read more about the differences between these datatypes in Pandas [here](https://pbpython.com/pandas_dtypes.html).

In [10]:
for name in standard_summary.columns:
    # only change the datatype for the Area columns
    if "Area" in name:
        standard_summary[name] = standard_summary[name].astype(int)

#### Normalizing The Data

In order to create statistically accurate calibration curves from the standard data, we need to account for variations in the retention times and peak areas. There are many different ways to accomplish this, but for this experiment, we will use the data from the internal standard, d8-toluene, to "normalize" the data for the analytes.